In [16]:
import numpy as np
import scipy as sp
from cvxopt import matrix, solvers
solvers.options['show_progress'] = False

Data for the test

In [17]:
asset_class = ['Euro Bonds','US Bonds', 'Canada', 'France', 'Germany', 'Japan', 'UK', 'US']
mean = np.array([0.27, 0.25, 0.39, 0.88, 0.53, 0.88, 0.79, 0.71])
sd_percent = [1.56, 2.01, 5.50, 7.03, 6.22, 7.04, 6.01, 4.30]
sd = np.array([0.01*x for x in sd_percent])
corr = np.array([[1.00, 0.92, 0.33, 0.26, 0.28, 0.16, 0.29, 0.42], 
                 [0.92, 1.00, 0.26, 0.22, 0.27, 0.14, 0.25, 0.36], 
                 [0.33, 0.26, 1.00, 0.41, 0.30, 0.25, 0.58, 0.71],
                 [0.26, 0.22, 0.41, 1.00, 0.62, 0.42, 0.54, 0.44],
                 [0.28, 0.27, 0.30, 0.62, 1.00, 0.35, 0.48, 0.34],
                 [0.16, 0.14, 0.25, 0.42, 0.35, 1.00, 0.40, 0.22],
                 [0.29, 0.25, 0.58, 0.54, 0.48, 0.40, 1.00, 0.56],
                 [0.42, 0.36, 0.71, 0.44, 0.34, 0.22, 0.56, 1.00]])
sd_down = np.tile(sd,(8,1))
sd.shape = (8,1)
sd_right = np.tile(sd,(1,8))
cov = sd_down*corr*sd_right


Mean Variance Efficient Frontier Algorithm & Resampling Efficient Frontier Algorithm

In [114]:
# 51 portfolios equally spaced with respect to return
# mu, sigma should be np.ndarray
def MVEF(mu, sigma, k=51):
    
    n = len(mu)
    c = matrix(mu)   
    h = matrix(-np.zeros(n))
    G = matrix(-np.identity(n))
    b = matrix(1.0)
    A = matrix(np.ones(n)).T

    # find r_min, r_max:
    r_min_solver = solvers.lp(c,G,h,A,b)
    r_min_weight = np.array(r_min_solver['x'])
    r_min = r_min_weight.T.dot(mu)
    
    r_max_solver = solvers.lp(-c,G,h,A,b)
    r_max_weight = np.array(r_max_solver['x'])
    r_max = r_max_weight.T.dot(mu)
    
    assert(r_min < r_max)
    
    r_rank = np.linspace(r_min,r_max,k)
    
    # build efficient frontier:
    ef_sd = []
    opt_weight = []
    G_hat = -matrix(np.vstack((mu.T,
                              np.identity(n))))
    for r in r_rank:
        P = 2*matrix(sigma)
        q = matrix(np.zeros(n))
        zero = np.zeros(n)
        zero.shape = (n,1)
        h_hat = -matrix(np.vstack((np.array([r]),zero)))
        ef_solver = solvers.qp(P,q,G_hat,h_hat,A,b)
        #ef_solver.options['show_progress'] = False
        weight = np.array(ef_solver['x'])
        opt_weight.append(weight)        
        ef_sigma = np.sqrt(weight.T.dot(sigma.dot(weight))[0,0])
        ef_sd.append(ef_sigma)
        
    return ef_sd, r_rank, opt_weight

def REF(mu, sigma, months=18*12, k=51, m=500):
    size = len(mu)
    REF_weights = [np.zeros((size,1)) for i in range(k)]
    REF_mean = []
    REF_sd = []
    for i in range(m):
        sample = np.random.multivariate_normal(mu, sigma, size).T
        sample_cov = np.cov(sample)
        sample_mean = np.mean(sample, 1)
        ref_sd, ref_r, ref_opt_weights = MVEF(sample_mean, sample_cov, k)
        for weight_REF, weight_ref in zip(REF_weights, ref_opt_weights):
            weight_REF += weight_ref
    for weight_REF in REF_weights:
        weight_REF /= m
        REF_mean.append(weight_REF.T.dot(mu)[0])
        REF_sd.append(np.sqrt(weight_REF.T.dot(sigma.dot(weight_REF))[0,0]))
    return REF_sd, REF_mean, REF_weights 
    
        
        

Result and Plot

In [112]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly 
plotly.tools.set_credentials_file(username='GY400', api_key='IqLG54Iong7FLeX17uh6')

true_ef_sd, true_ef_mean, true_opt_weight = MVEF(mean, cov)
ref_sd, ref_mean, ref_weight = REF(mean, cov)

#print(true_ef_sd)

#print(ref_sd)
#print(ref_mean)
print(ref_weight[25])
print(true_opt_weight[25])

efficient_frontier = go.Scatter(
                x = true_ef_sd,
                y = true_ef_mean,
                mode = 'markers'
                )

resampling_ef = go.Scatter(
             x = ref_sd,
             y = ref_mean,
             mode = 'markers'
             )

py.iplot([efficient_frontier, resampling_ef])
#py.iplot([resampling_ef])

[[ 0.32872547]
 [ 0.06617121]
 [ 0.01473561]
 [ 0.08192781]
 [ 0.04723716]
 [ 0.15060952]
 [ 0.09169677]
 [ 0.21889643]]
[[  4.17489465e-01]
 [  1.46693607e-06]
 [  1.73936115e-07]
 [  5.80298886e-02]
 [  3.86855184e-06]
 [  1.57623427e-01]
 [  2.54469810e-02]
 [  3.41404730e-01]]


Edge of REF: the Simulation Proof

In [119]:
size = len(mean)
est_sample = np.random.multivariate_normal(mean, cov, size).T
est_cov = np.cov(sample)
est_mean = np.mean(sample, 1)

assert(est_cov.shape == (size,size))

mv_sd, mv_mean, mv_weights = MVEF(est_mean, est_cov)
r_sd, r_mean, r_weights = REF(est_mean, est_cov, m=100)

est_mv_ef_mean = []
est_mv_ef_sd = []
est_r_ef_mean = []
est_r_ef_sd = []

for weight_mv, weight_r in zip(mv_weights, r_weights):
    est_mv_ef_mean.append(weight_mv.T.dot(mean)[0])
    est_mv_ef_sd.append(np.sqrt(weight_mv.T.dot(cov.dot(weight_mv))[0,0]))
    est_r_ef_mean.append(weight_r.T.dot(mean)[0])
    est_r_ef_sd.append(np.sqrt(weight_r.T.dot(cov.dot(weight_r))[0,0]))

est_mv_ef = go.Scatter(
                x = est_mv_ef_sd,
                y = est_mv_ef_mean,
                name = 'mv',
                mode = 'markers'
                )

est_resampling_ef = go.Scatter(
             x = est_r_ef_sd,
             y = est_r_ef_mean,
             name = 'resampling',
             mode = 'markers'
             )

py.iplot([est_mv_ef, est_resampling_ef])
